In [1]:

import argparse
import http.client
import httplib2
import os
import random
import time
import datetime
from datetime import datetime, timedelta
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
import os
from dotenv import load_dotenv
from sqlalchemy import *
import datetime
from Google import Create_Service
import re
from IPython.display import Image
load_dotenv()
from StreamMaster import *


In [93]:
engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER_NAME']}:{os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" )


clips_folder = os.environ['CLIPS_FOLDER']
mobile_folder = os.environ['MOBILE_FOLDER']
fullscreen_folder = os.environ['FULLSCREEN_FOLDER']
ml_folder = os.environ['ML_FOLDER']
assets = os.environ['ASSETS']

CLIENT_SECRET_FILE = 'client_secrets.json'
YT_SECRET_FILE = 'client_secrets_yt.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
# recommend pulling as much access as possible since the Create_Service function will create a pickle token of your authentication allow single authentication for extended period.
SCOPES = ["https://www.googleapis.com/auth/youtube.upload", 'https://www.googleapis.com/auth/youtube']

# Code that initiates connection to the Google API - pulled from Google.py. Difference from using Google's built-in service is the following script creates convenience of storing pickle with necessary authentication
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

client_secrets.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/youtube'],)
['https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/youtube']
youtube service created successfully


In [96]:
def get_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
    response = client_library_function(
        **kwargs
        ).execute()
    print(response)

In [91]:
service.reports().query()

AttributeError: 'Resource' object has no attribute 'reports'

In [97]:
youtubeAnalytics = get_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=161873373944-rhokqgo1lmbgjcn75a2vs64qr8brpecv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=WxiPhhRIAtRe95GfhlOjGRKSSCgIUF&prompt=consent&access_type=offline


ValueError: Please supply either code or authorization_response parameters.

In [85]:
execute_api_request(
      service.reports().query,
      ids='channel==MINE',
      metrics='estimatedMinutesWatched,views,likes,subscribersGained',
      dimensions='day',
      sort='day'
  )

AttributeError: 'Resource' object has no attribute 'reports'

In [3]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None

In [7]:
for playlist in playlists['items']:
    if "Shorts" in playlist['snippet']['title']:
        style = "m"
        print(playlist['snippet']['title'])
    else:
        style = "f"

All Shorts


In [10]:
def completeThumbnails(playlist, default_mode = 'default'):
    default = playlist['snippet']['thumbnails'][default_mode]
    checks = ['default', 'medium', 'high', 'standard']
    for chck in checks:
        if not bool(playlist['snippet']['thumbnails'].get(chck)):
            playlist['snippet']['thumbnails'][chck] = default
    return playlist

In [17]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None

mapped_as_full_counts = {}
mapped_full_vids = {}
with Session(engine) as session:
    for title in session.query(Clip_Tracker.title).where(Clip_Tracker.published == PublishingStatus.f).order_by(Clip_Tracker.view_count.desc()).all():
        mapped_full_vids[title[0]] = []
        mapped_as_full_counts[title[0]] = 0
shorts_pattern = f"[sS]hort" # check for YT short videos
with Session(engine) as session: # initialize session to sql using context manager
    for playlist in playlists['items']: # loop through the differen playlists
        playlist = completeThumbnails(playlist)
        for key, thumbnail in playlist['snippet']['thumbnails'].items(): # first ensure that thumbnails are mapped in thumbnail table
            if not bool(session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()):
                session.add(Thumbnails(asset=thumbnail['url'], width = thumbnail['width'], height=thumbnail['height']))
                session.commit()
            # map the thumbnail table ids to the playlist dictionary
            playlist['snippet']['thumbnails'][key]['id'] = session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()[0]

        # if the playlist id is not in the table, add it
        if not bool(session.query(PlayList_yt).where(PlayList_yt.id == playlist['id']).first()):
            session.add(PlayList_yt(id=playlist['id'], title=playlist['snippet']['title'], 
            tb_default = playlist['snippet']['thumbnails']['default']['id'], 
            tb_medium = playlist['snippet']['thumbnails']['medium']['id'],
            tb_high = playlist['snippet']['thumbnails']['high']['id'], 
            tb_standard = playlist['snippet']['thumbnails']['standard']['id']
            ))
            session.commit()

        # else, update the table with the current values if it's already in the table - this is to capture if thumbnails or titles have changed
        else:
            session.execute(
                update(PlayList_yt).where(PlayList_yt.id == playlist['id']).values(dict(title=playlist['snippet']['title'], 
            tb_default = playlist['snippet']['thumbnails']['default']['id'], 
            tb_medium = playlist['snippet']['thumbnails']['medium']['id'],
            tb_high = playlist['snippet']['thumbnails']['high']['id'], 
            tb_standard = playlist['snippet']['thumbnails']['standard']['id']))
            )
            session.commit()
        if "Shorts" in playlist['snippet']['title']:
            style = VideoStyle.m
        else:
            style = VideoStyle.f
        # next find all the videos in each playlist
        playlist_Items = service.playlistItems().list(part="id,snippet", playlistId=playlist['id']).execute()
            
        nextPage = playlist_Items['nextPageToken']
        while nextPage != None:
            seed_list = service.playlistItems().list(part="id,snippet", playlistId=playlist['id'], pageToken = nextPage).execute()
            nextPage = seed_list['nextPageToken']
            playlist_Items['items'] = playlist_Items['items'] + seed_list['items']
        for video in playlist_Items['items']:
            try:
                video = completeThumbnails(video)
                title = video['snippet']['title']
                description = video['snippet']['description']
                    # check whether the title or description have the word "short" in it
                # combine the tags into a  single string due to datatype limitations in database - leave as array if your database can store arrays
                tags = " | ".join(service.videos().list(part='snippet,contentDetails', id=video['snippet']['resourceId']['videoId']).execute()['items'][0]['snippet']['tags'])
                

                # to capture titles with title conventon used in yt_videoupload.py
                title_components = title.split("|")
                if len(title_components) > 1:
                    titleName = title_components[-1].strip()
                
                else: # incase title does not follow current convention
                    titleName = title
                
                try:
                    mapped_as_full_counts[titleName] += 1
                    mapped_full_vids[titleName].append(style)
                except:
                    pass

            
                # find clip and original filename in clips database
                for clip_id, filename in session.query(Clip_Tracker.id, Clip_Tracker.video_name).where(Clip_Tracker.title == titleName).all():

                    if not bool(clip_id):
                        print(title)
                    else:
                        # loop through the thumbnail images to ensure they are in the thumbnail database
                        for key, thumbnail in video['snippet']['thumbnails'].items():
                            if not bool(session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()):
                                session.add(Thumbnails(asset=thumbnail['url'], width = thumbnail['width'], height=thumbnail['height']))
                                session.commit()
                            video['snippet']['thumbnails'][key]['id'] = session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()[0]
                        # VideoID is not in the Video_yt table - add it!    
                        if not bool(session.query(Video_yt).where(Video_yt.id == video['snippet']['resourceId']['videoId']).first()):
                            session.add( Video_yt(id = video['snippet']['resourceId']['videoId'], 
                            clip_id=clip_id, 
                            title = title,
                            filename = filename,
                            style = style,
                            description = description,
                            tb_default = video['snippet']['thumbnails']['default']['id'], 
                            tb_medium = video['snippet']['thumbnails']['medium']['id'],
                            tb_high = video['snippet']['thumbnails']['high']['id'], 
                            tb_standard = video['snippet']['thumbnails']['standard']['id'],
                            tags = tags
                            
                            )
                            )
                            session.commit()
                        # If it's already in there, update our record of current descriptiption / thumbnails.
                        else:
                            if style == VideoStyle.m:
                                print(video)
                            session.execute(
                                update(Video_yt).where(Video_yt.id == video['snippet']['resourceId']['videoId']).values(
                                    dict(
                                        clip_id=clip_id, 
                                        title = title,
                                        filename = filename,
                                        style = style,
                                        description = description,
                                        tb_default = video['snippet']['thumbnails']['default']['id'], 
                                        tb_medium = video['snippet']['thumbnails']['medium']['id'],
                                        tb_high = video['snippet']['thumbnails']['high']['id'], 
                                        tb_standard = video['snippet']['thumbnails']['standard']['id'],
                                        tags = tags
                                    )
                                )
                                )
                            session.commit()
                        if not bool(session.query(yt_vid_pl_mapper).where(yt_vid_pl_mapper.c.yt_video_id == video['snippet']['resourceId']['videoId']).where(yt_vid_pl_mapper.c.yt_playlist_id ==playlist['id']).first()):
                            session.execute(insert(yt_vid_pl_mapper).values({"yt_playlist_id": playlist['id'], 'yt_video_id': video['snippet']['resourceId']['videoId']}))
            except Exception as e:
                print("failed", e)

KeyError: 'nextPageToken'

In [74]:
playlist_Items['items'][0]['snippet']

{'publishedAt': '2022-04-16T05:58:42Z',
 'channelId': 'UCnazyWXANZqGB7Yj2slN8gg',
 'title': "PS5 | Demon's Souls Gameplay |  I mean...",
 'description': 'Follow me on Twitch for weekly live gameplay!\nhttps://twitch.tv/theroundwon\n\nSubscribe to get notified of new videos: \nhttps://www.youtube.com/channel/UCnazyWXANZqGB7Yj2slN8gg?sub_confirmation=1',
 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/default.jpg',
   'width': 120,
   'height': 90},
  'medium': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'high': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/hqdefault.jpg',
   'width': 480,
   'height': 360},
  'standard': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/sddefault.jpg',
   'width': 640,
   'height': 480},
  'maxres': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/maxresdefault.jpg',
   'width': 1280,
   'height': 720}},
 'channelTitle': 'TheRoundWon',
 'playlistId': 'PL3KuKEx0WXaaHu9tMszeGhrjEBH3dJaKe',
 '

In [77]:
with Session(engine) as session:
    print(session.query(yt_vid_pl_mapper).where(yt_vid_pl_mapper.c.yt_playlist_id == playlist['id']).filter(~yt_vid_pl_mapper.c.yt_video_id.in_([video['snippet']['resourceId']['videoId'] for video in playlist_Items['items']])).all())

[]


In [35]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None
all_items = {}
for playlist in playlists['items']: 
    playlist_Items = service.playlistItems().list(part="id,snippet", playlistId=playlist['id']).execute()
    all_items[playlist['id']] = {'playlist_items': None, "videos": []}
    all_items[playlist['id']]['playlist_items'] = playlist_Items
    if bool(playlist_Items.get('nextPageToken')):
        nextPage = playlist_Items['nextPageToken']
    else:
        nextPage = None
    while nextPage != None:
        seed_list = service.playlistItems().list(part="id,snippet", playlistId=playlist['id'], pageToken = nextPage).execute()
        if bool(seed_list.get('nextPageToken')):
            nextPage = seed_list['nextPageToken']
        else:
            nextPage = None
        playlist_Items['items'] = playlist_Items['items'] + seed_list['items']
        for video in playlist_Items['items']:
            all_items[playlist['id']]['videos'].append(service.videos().list(part='snippet,contentDetails', id=video['snippet']['resourceId']['videoId']).execute())

In [63]:
for key, value in all_items.items():
    for value2 in value['videos']:
        if bool(value2['items']):
            if value2['items'][0]['kind'] != 'youtube#video':
                # if bool()
                print(value2)

In [59]:
value2['items']

[{'kind': 'youtube#video',
  'etag': '7SPI_yqIPvcT_2GRQJrEy6Sv_xc',
  'id': 'fFLk8tEJE5Y',
  'snippet': {'publishedAt': '2022-06-13T03:44:09Z',
   'channelId': 'UCnazyWXANZqGB7Yj2slN8gg',
   'title': 'iOS | Clash Royale Gameplay | Barrel talk',
   'description': 'Follow me on Twitch for weekly live gameplay\nhttps://twitch.tv/theroundwon\nCheck out the original clip on Twitch! https://clips.twitch.tv/FurtiveEntertainingRaisinVoteYea-Xsn8Z_uRRg3BS29l\n\n#ClashRoyale #videogames #followme #twitch #casualplay #TheRoundWon #iOS\nGame play Footage from Clash Royale\nBig Thanks to TheSquareWon for capturing the footage!\n\nSubscribe to get notified of new videos:\nhttps://www.youtube.com/channel/UCnazyWXANZqGB7Yj2slN8gg?sub_confirmation=1',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/fFLk8tEJE5Y/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/fFLk8tEJE5Y/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'u

In [25]:
with Session(engine) as session:
    print(session.query(Video_yt.title, Video_yt.))

'AbstemiousWimpyRadicchioANELE-cK2OfUlqSllNC9sX'

In [72]:
playlist_Items['items']

[{'kind': 'youtube#playlistItem',
  'etag': 'Zl2bRDeSWpKDFSATxajNCJ9eo2I',
  'id': 'UEwzS3VLRXgwV1hhYUh1OXRNc3plR2hyakVCSDNkSmFLZS41NkI0NEY2RDEwNTU3Q0M2',
  'snippet': {'publishedAt': '2022-04-16T05:58:42Z',
   'channelId': 'UCnazyWXANZqGB7Yj2slN8gg',
   'title': "PS5 | Demon's Souls Gameplay |  I mean...",
   'description': 'Follow me on Twitch for weekly live gameplay!\nhttps://twitch.tv/theroundwon\n\nSubscribe to get notified of new videos: \nhttps://www.youtube.com/channel/UCnazyWXANZqGB7Yj2slN8gg?sub_confirmation=1',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/Ene3KibtrVo/sddefault.jpg',
     'width': 640,
     'height': 480},
   

In [70]:
with Session(engine) as session:
    for clip_id in session.query(Clip_Tracker.id).where(Clip_Tracker.published == PublishingStatus.f).all():
        rows = session.query(Video_yt.title, Video_yt.style).where(Video_yt.clip_id == clip_id[0]).all()
        if len(rows) < 2 and bool(rows):
            session.query(Clip_Tracker).where(Clip_Tracker.id == clip_id[0]).update(values = {'published': invers_mapper[rows[0][1]]})
            session.commit()

            print(rows[0])

('PS5 | Overcooked! All You Can Eat Gameplay | Spicy Square People', <VideoStyle.m: 1>)


In [68]:
rows[0]

('PS5 | Overcooked! All You Can Eat Gameplay | Spicy Square People', <VideoStyle.m: 1>)